<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/Stock_NeurIPS2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Task Description](#0)
* [2. Install Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. A List of Python Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download and Preprocess Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5. Build Market Environment in OpenAI Gym-style](#4)  
    * [5.1. Data Split](#4.1)  
    * [5.3. Environment for Training](#4.2)    
* [6. Train DRL Agents](#5)
* [7. Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
  

<a id='0'></a>
# Part 1. Task Discription

We train a DRL agent for stock trading. This task is modeled as a Markov Decision Process (MDP), and the objective function is maximizing (expected) cumulative return.

We specify the state-action-reward as follows:

* **State s**: The state space represents an agent's perception of the market environment. Just like a human trader analyzing various information, here our agent passively observes many features and learns by interacting with the market environment (usually by replaying historical data).

* **Action a**: The action space includes allowed actions that an agent can take at each state. For example, a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying. When an action operates multiple shares, a ∈{−k, ..., −1, 0, 1, ..., k}, e.g.. "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* **Reward function r(s, a, s′)**: Reward is an incentive for an agent to learn a better policy. For example, it can be the change of the portfolio value when taking a at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively


**Market environment**: 30 consituent stocks of Dow Jones Industrial Average (DJIA) index. Accessed at the starting date of the testing period.


The data for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Install Python Packages

<a id='1.1'></a>
## 2.1. Install packages


In [ ]:
!pip install Ta-Lib

In [ ]:
pip install Ta-Lib -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
pip install git+https://github.com/quantopian/pyfolio

In [ ]:
## install finrl library
!pip install wrds -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install swig -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git 


<a id='1.2'></a>
## 2.2. A list of Python packages 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import datetime
import torch
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as ts
# matplotlib.use('Agg')
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader


%matplotlib inline
#from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading_adjust import StockTradingEnv
from finrl.agents.stablebaselines3.models_add import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../mha-gru-drl")

import itertools

<a id='1.4'></a>
## 2.4. Create Folders

In [3]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



<a id='2'></a>
# Part 3. Download Data
Yahoo Finance provides stock data, financial news, financial reports, etc. Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** in FinRL-Meta to fetch data via Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).



-----
class YahooDownloader:
    Retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()


In [ ]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

In [22]:
TRAIN_START_DATE = '2017-03-02'
TRAIN_END_DATE = '2020-02-27'
TRADE_START_DATE = '2020-02-28'
TRADE_END_DATE = '2022-12-30'

In [5]:
import tushare as ts
ts.set_token('df3c6a8e252ff736ca08a3364022e4340b68485bf9f9d3cea4c94f21')
pro = ts.pro_api()

In [6]:
#选择上证50指数的成分股
df_index = pro.index_weight(index_code='000016.sh', start_date='20121010', end_date='20220228')
# df_index.to_csv('SSE_50_index_weight.csv',index=0)

In [7]:
#选择某个时间点的上证50指数成分股作为参考跟踪股票池
select_date = '20160229' #自定义的时间点为每个月的月末
df_select = df_index[df_index['trade_date']==select_date]
sort_SSEindex = df_select['con_code'].unique()
print(sort_SSEindex,len(sort_SSEindex))

['601318.SH' '600016.SH' '601166.SH' '600000.SH' '600036.SH' '601328.SH'
 '601288.SH' '600030.SH' '600519.SH' '600837.SH' '601169.SH' '601398.SH'
 '601766.SH' '600887.SH' '601668.SH' '601601.SH' '601988.SH' '600104.SH'
 '600048.SH' '601818.SH' '601989.SH' '600015.SH' '600028.SH' '600637.SH'
 '601688.SH' '601390.SH' '600999.SH' '600518.SH' '601006.SH' '601857.SH'
 '600050.SH' '601186.SH' '601628.SH' '601985.SH' '600795.SH' '600585.SH'
 '600893.SH' '601088.SH' '600010.SH' '601901.SH' '601211.SH' '601669.SH'
 '600111.SH' '601336.SH' '600109.SH' '600958.SH' '601998.SH' '601800.SH'
 '600018.SH' '600150.SH'] 50


In [8]:
#随机选择成分股中的k只股票
import random 
k = 50
selected_tics = random.sample(list(sort_SSEindex),k)
print(selected_tics)

['601800.SH', '600795.SH', '601998.SH', '600519.SH', '601988.SH', '601688.SH', '601989.SH', '600837.SH', '601336.SH', '600048.SH', '601901.SH', '600015.SH', '600999.SH', '601169.SH', '601668.SH', '601398.SH', '600111.SH', '600036.SH', '601628.SH', '600585.SH', '600000.SH', '600887.SH', '601669.SH', '600016.SH', '600104.SH', '601186.SH', '601328.SH', '601211.SH', '600518.SH', '601766.SH', '601390.SH', '600028.SH', '601818.SH', '601288.SH', '601601.SH', '601006.SH', '601985.SH', '601318.SH', '600893.SH', '601166.SH', '600030.SH', '600018.SH', '601857.SH', '601088.SH', '600150.SH', '600958.SH', '600637.SH', '600010.SH', '600050.SH', '600109.SH']


In [36]:
df_sz = pro.index_daily(ts_code = '000016.SH',start_date='2017-04-01',end_date='2023-12-01')
df_sz = df_sz.sort_values(['ts_code','trade_date'],ascending=True,ignore_index=True)
df_sz

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,000016.SH,20170103,2307.8917,2285.2729,2311.2926,2285.2729,2286.8984,20.9933,0.9180,25908920.0,2.544074e+07
1,000016.SH,20170104,2322.2059,2305.9117,2324.5577,2304.1733,2307.8917,14.3142,0.6202,26104403.0,2.718326e+07
2,000016.SH,20170105,2322.6843,2322.4239,2326.1847,2317.8853,2322.2059,0.4784,0.0206,26528712.0,2.656642e+07
3,000016.SH,20170106,2308.9363,2323.0064,2326.4342,2308.3694,2322.6843,-13.7480,-0.5919,26445124.0,2.728669e+07
4,000016.SH,20170109,2318.3365,2308.4423,2321.6920,2307.1634,2308.9363,9.4002,0.4071,26523074.0,2.590351e+07
...,...,...,...,...,...,...,...,...,...,...,...
1454,000016.SH,20221226,2607.5229,2617.1819,2625.8838,2604.8991,2615.8794,-8.3565,-0.3195,19812417.0,4.156645e+07
1455,000016.SH,20221227,2632.4526,2623.1749,2640.7395,2616.9293,2607.5229,24.9297,0.9561,21814661.0,3.985271e+07
1456,000016.SH,20221228,2638.3055,2629.9846,2646.7069,2620.1185,2632.4526,5.8529,0.2223,23171189.0,4.165666e+07
1457,000016.SH,20221229,2622.0716,2622.1933,2627.4310,2605.0487,2638.3055,-16.2339,-0.6153,22776667.0,4.125150e+07


In [45]:
df_sz.loc[2,'ema'].values.tolist()

AttributeError: 'numpy.float64' object has no attribute 'values'

In [49]:
df_sz.ema

0          0.000000
1          0.000000
2          0.000000
3          0.000000
4          0.000000
           ...     
1454    2611.166438
1455    2612.539738
1456    2614.202046
1457    2614.709759
1458    2616.034832
Name: ema, Length: 1459, dtype: float64

In [37]:
import talib as ta
df_sz['ema'] = ta.EMA(df_sz["close"], timeperiod=30)
df_sz = df_sz.fillna(0)

In [51]:
df_sz = df_sz[-1411:]

In [52]:
df_sz.to_csv('1230_sz.csv',index=None)

In [20]:
TRADE_END_DATE

'2023-12-01'

In [ ]:

selected_tics = ['600104.SH','600050.SH','600048.SH','600036.SH','600031.SH','600030.SH','600028.SH','600016.SH','600009.SH','600000.SH']


In [9]:
#Download随机选择的k只股票数据
df_ts =pd.DataFrame()
for c in selected_tics:
    temp=pro.daily(ts_code=c,start_date=TRAIN_START_DATE,end_date=TRADE_END_DATE)
    df_ts=pd.concat([df_ts,temp])
print(df_ts)

        ts_code trade_date   open   high    low  close  pre_close  change  \
0     601800.SH   20221230   8.01   8.09   7.92   8.05       7.98    0.07   
1     601800.SH   20221229   8.13   8.16   7.91   7.98       8.11   -0.13   
2     601800.SH   20221228   8.18   8.29   8.08   8.11       8.24   -0.13   
3     601800.SH   20221227   8.22   8.35   8.13   8.24       8.19    0.05   
4     601800.SH   20221226   8.16   8.23   8.02   8.19       8.19    0.00   
...         ...        ...    ...    ...    ...    ...        ...     ...   
1444  600109.SH   20170109  13.14  13.24  13.09  13.20      13.18    0.02   
1445  600109.SH   20170106  13.36  13.38  13.16  13.18      13.37   -0.19   
1446  600109.SH   20170105  13.37  13.42  13.30  13.37      13.39   -0.02   
1447  600109.SH   20170104  13.28  13.42  13.23  13.39      13.32    0.07   
1448  600109.SH   20170103  13.05  13.51  13.02  13.32      13.03    0.29   

      pct_chg        vol      amount  
0      0.8772  448578.31  359841.007

In [ ]:
df_ts = pd.read_csv('10_test_628.csv')

In [ ]:
df_ts.to_csv('10_test_628.csv',index=None)

In [13]:
max_len = max(df_ts['ts_code'].value_counts())

In [10]:
def processed_date_range(df,selected_tics,k):
    longth = []
    tic_list = []
    for tic in selected_tics:
        temp_df = df[df['ts_code'] == tic]
        temp_long = temp_df['trade_date'].iloc[-1]
        longth.append(temp_long)
        tic_list.append(tic)
#         print(tic_list,longth)
    minlong = max(longth)
    date_unique = [date for date in df['trade_date'].unique() if date >= minlong]
    full_date_range = pd.DataFrame(date_unique,columns=['trade_date'])
    
    return full_date_range
#     date_unique = df['trade_date'].unique()

In [11]:
k=50
full_date_range = processed_date_range(df_ts,selected_tics,k)

In [12]:
#遍历扩充股票的完整交易日期
def add_exchange_calendars(df,full_date_range,selected_tics):
    count = 0
    merge_df = []
    for tic in selected_tics:
        temp_df = df[df['ts_code'] == tic]
        temp_full_date_range = full_date_range
        temp_df = temp_df.set_index('trade_date')
        temp_full_date_range = temp_full_date_range.set_index('trade_date')
        temp_df = pd.merge(temp_full_date_range,temp_df,how='left',left_index=True,right_index=True)
        temp_df = temp_df.reset_index().sort_values('trade_date',ascending=True)
        temp_df = temp_df.fillna({
            'amount' : 0,
            'vol': 0,
            'pct_chg': 0,
            'change':0,
            'ts_code': tic
        })
        temp_df['filled'] = temp_df['close'].isna().astype(int)
#         print(temp_df,temp_df['filled'].unique())
        
        for i in range(len(temp_df)):
            if pd.isna(temp_df.loc[i,'close']):
                j = i -1
                while pd.isna(temp_df.loc[j,'close']):
                    j = j - 1
                if j > 0 :
                    temp_df.loc[i,['close','open','high','low','pre_close']] = temp_df.loc[j,'close']
                    count+=1
        merge_df.append(temp_df)
    merged_df = pd.concat(merge_df,ignore_index = True)
    print(count)
    return merged_df

merged_df =  add_exchange_calendars(df_ts, full_date_range, selected_tics)
# merged_df = merged_df.rename(columns={'trade_date':'date','ts_code':'tic','vol':'volume'})
merged_df = merged_df[['ts_code','trade_date','open','high','low','close','pre_close','change','pct_chg','vol','amount','filled']]
print(merged_df)

669
         ts_code trade_date   open   high    low  close  pre_close  change  \
0      601800.SH   20170103  15.13  15.39  15.13  15.29      15.19    0.10   
1      601800.SH   20170104  15.24  15.60  15.11  15.45      15.29    0.16   
2      601800.SH   20170105  15.40  15.65  15.18  15.42      15.45   -0.03   
3      601800.SH   20170106  15.43  15.90  15.32  15.61      15.42    0.19   
4      601800.SH   20170109  15.41  15.79  15.19  15.75      15.61    0.14   
...          ...        ...    ...    ...    ...    ...        ...     ...   
72945  600109.SH   20221226   8.68   8.73   8.48   8.52       8.69   -0.17   
72946  600109.SH   20221227   8.60   8.72   8.55   8.68       8.52    0.16   
72947  600109.SH   20221228   8.64   8.65   8.55   8.57       8.68   -0.11   
72948  600109.SH   20221229   8.54   8.72   8.51   8.62       8.57    0.05   
72949  600109.SH   20221230   8.64   8.73   8.64   8.70       8.62    0.08   

       pct_chg        vol      amount  filled  
0       0.6

In [14]:
import talib as ta
merged_df['ema'] = ta.EMA(merged_df["close"], timeperiod=30)
merged_df = merged_df.fillna(0)

In [16]:
merged_df

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,filled,ema
0,601800.SH,20170103,15.13,15.39,15.13,15.29,15.19,0.10,0.6600,247541.17,378230.308,0,0.000000
1,601800.SH,20170104,15.24,15.60,15.11,15.45,15.29,0.16,1.0500,262239.57,404372.419,0,0.000000
2,601800.SH,20170105,15.40,15.65,15.18,15.42,15.45,-0.03,-0.1900,239180.84,367547.886,0,0.000000
3,601800.SH,20170106,15.43,15.90,15.32,15.61,15.42,0.19,1.2300,392150.44,615064.151,0,0.000000
4,601800.SH,20170109,15.41,15.79,15.19,15.75,15.61,0.14,0.9000,284037.78,440851.265,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72945,600109.SH,20221226,8.68,8.73,8.48,8.52,8.69,-0.17,-1.9563,255110.22,217941.744,0,8.769376
72946,600109.SH,20221227,8.60,8.72,8.55,8.68,8.52,0.16,1.8779,247763.62,214491.273,0,8.763610
72947,600109.SH,20221228,8.64,8.65,8.55,8.57,8.68,-0.11,-1.2673,113096.03,97101.726,0,8.751119
72948,600109.SH,20221229,8.54,8.72,8.51,8.62,8.57,0.05,0.5834,147612.50,127139.208,0,8.742660


In [17]:
merged_df.to_csv('1230_merged_ema.csv',index=None)

In [ ]:
processed_df = merged_df

In [ ]:
processed_df1 = processed_df
processed_df1 = processed_df1.sort_values(['ts_code','trade_date'],ascending=True,ignore_index=True)
processed_df1['trade_date'] = pd.to_datetime(processed_df1['trade_date'])
processed_df1

In [ ]:
# #检验50只股票的信息缺省的时间点和个数
# dimatch_num = 0
# dimatch_date =[]
# for c in df_ts1['trade_date'].unique():
#     temp = df_ts1[df_ts1['trade_date'] == c]
#     if len(temp) != 50:
#         dimatch_num +=1
#         dimatch_date.append(c)
# print(dimatch_num,dimatch_date)

In [ ]:
type(full_date_range)

In [ ]:
# date_unique = df['date'].unique()
# date_sum = len(date_unique)
# count = 0
# for date in date_unique:
#     temp_df = df[df['trade_date'] == date]
# #     if len(temp_df) = k:
# #         count +=1
# #     elif len(temp_df) < int(0.9*k):
# #         date_unique.remove(date)
# #     else:
#     difference_tic = temp_df['ts_code'].tolist().difference(selected_tics)
#     missing_tic.append(date,difference_tic)

In [ ]:
# #遍历扩充股票的完整交易日期
# def add_exchange_calendars(df,full_date_range,selected_tics):
#     merge_df = []
#     for tic in selected_tics:
#         temp_df = df[df['ts_code'] == tic]
#         temp_full_date_range = full_date_range
#         temp_df = temp_df.set_index('trade_date')
#         temp_full_date_range = temp_full_date_range.set_index('trade_date')
#         temp_df = pd.merge(temp_full_date_range,temp_df,how='left',left_index=True,right_index=True)
#         temp_df = temp_df.reset_index().sort_values('trade_date',ascending=True)
#         temp_df = temp_df.fillna({
#             'amount' : 0,
#             'vol': 0,
#             'pct_chg': 0,
#             'change':0,
#             'ts_code': tic
#         })
#         for i in range(len(temp_df)):
#             if pd.isna(temp_df.loc[i,'close']):
#                 j = i -1
#                 while pd.isna(temp_df.loc[j,'close']):
#                     j = j - 1
#                 if j > 0 :
#                     temp_df.loc[i,['close','open','high','low','pre_close']] = temp_df.loc[j,'close']
#         merge_df.append(temp_df)
#     merged_df = pd.concat(merge_df,ignore_index = True)
#     return merged_df

# merged_df =  add_exchange_calendars(df_ts,full_date_range,selected_tics)
# merged_df = merged_df.rename(columns={'trade_date':'date','ts_code':'tic','vol':'volume'})
# merged_df = merged_df[['tic','date','open','high','low','close','pre_close','change','pct_chg','volume','amount']]
# print(merged_df)

In [ ]:
# # 筛选exchange_calender中每个交易节点缺失情况
# def processed_date(df,selected_tics,k):
#     date_unique = list(df['trade_date'].unique())
#     date_sum = len(date_unique)
#     count = 0
#     processe_df =[]
#     processed_df = pd.DataFrame()
#     for date in date_unique:
#         temp_df = df[df['trade_date'] == date]
#         if len(temp_df) < int(0.9*k):
#             date_unique.remove(date)
#         elif len(temp_df) >= int(0.9*k) and len(temp_df) < k:
#             missing_tic = set(selected_tics).difference(set(temp_df['ts_code'].tolist()))
# #             print(missing_tic)
#             for m_tic in missing_tic:
#                 temp_dict = {'ts_code':m_tic,'trade_date':date,'open':np.nan, 'high':np.nan,'low':np.nan, 'close':np.nan, 'pre_close':np.nan, 'change':np.nan, 'pct_chg':np.nan, 'volume':np.nan, 'amount':np.nan}
#                 new_data = pd.DataFrame.from_dict(temp_dict,orient='index').T
#                 temp_df = temp_df.append(new_data)
# #             print(len(temp_df))
#             processe_df.append(temp_df)
#         else :
#             count += 1
#             processe_df.append(temp_df)
#         processed_df = pd.concat(processe_df,ignore_index = True)       
#     processed_df = processed_df.sort_values('trade_date',ascending=True).reset_index()
#     processed_df = processed_df.drop('index',axis=1)
#     return processed_df,date_unique


In [ ]:
# # 筛选exchange_calender中每个交易节点缺失情况(全部补充0)
# def processed_date(df,selected_tics,k):
#     date_unique = list(df['trade_date'].unique())
#     date_sum = len(date_unique)
#     count = 0
#     processe_df =[]
#     processed_df = pd.DataFrame()
#     for date in date_unique:
#         temp_df = df[df['trade_date'] == date]
#         if len(temp_df) != k:
#             missing_tic = set(selected_tics).difference(set(temp_df['ts_code'].tolist()))
# #             print(missing_tic)
#             for m_tic in missing_tic:
#                 temp_dict = {'ts_code':m_tic,'trade_date':date,'open':0, 'high':0,'low':0, 'close':0, 'pre_close':0, 'change':0, 'pct_chg':0, 'volume':0, 'amount':0}
#                 new_data = pd.DataFrame.from_dict(temp_dict,orient='index').T
#                 temp_df = temp_df.append(new_data)
#             processe_df.append(temp_df)
#         else :
#             processe_df.append(temp_df)
#         processed_df = pd.concat(processe_df,ignore_index = True)       
#     processed_df = processed_df.sort_values('trade_date',ascending=True).reset_index()
#     processed_df = processed_df.drop('index',axis=1)
#     return processed_df,date_unique

In [ ]:
# # 筛选exchange_calender中每个交易节点缺失(激进)
# def processed_date(df,selected_tics,k):
#     date_unique = list(df['trade_date'].unique())
#     date_sum = len(date_unique)
#     count = 0
#     processe_df =[]
#     processed_df = pd.DataFrame()
#     for date in date_unique:
#         temp_df = df[df['trade_date'] == date]
#         if len(temp_df) == k:
#             processe_df.append(temp_df)
#         else :
#             date_unique.remove(date)
#         processed_df = pd.concat(processe_df,ignore_index = True)       
#     processed_df = processed_df.sort_values('trade_date',ascending=True).reset_index()
#     processed_df = processed_df.drop('index',axis=1)
#     return processed_df,date_unique
        

In [ ]:
# processed_df,date_unique = processed_date(df_ts,selected_tics,k)
# print(processed_df) 

In [ ]:
# processed_df1 = processed_df.drop('volume',axis=1)
# processed_df1 = processed_df1.sort_values(['ts_code','trade_date'],ascending=True,ignore_index=True)
# processed_df1['trade_date'] = pd.to_datetime(processed_df1['trade_date'])
# processed_df1

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=False,
                    use_turbulence=False,
                    user_defined_feature = False)

In [ ]:
processed_df3 = processed_df1.rename(columns={'ts_code':'tic','trade_date':'date','vol':'volume'})
processed_df3 = fe.preprocess_data(processed_df3)
processed_df3 = processed_df3.fillna(method="ffill").fillna(method="bfill")
processed_df3 = processed_df3.sort_values(['tic','date'],ascending=True).reset_index()

In [ ]:
processed_df1

In [ ]:
processed_df3

In [ ]:
def delaydate(dela,full_list):
    grouped = processed_df3.groupby('tic')
    filtered_df = pd.DataFrame()
    for name, group in grouped:
        # 按'trade_date'升序排序  True=升序
        group = group.sort_values('date', ascending=True)

        # 丢弃前48个日期的数据
        group = group.iloc[dela:]
        filtered_df = filtered_df.append(group)
    
    filtered_df = filtered_df.reset_index(drop=True)
    filtered_df = filtered_df.sort_values(['tic','date'],ascending=True,ignore_index=True)
    return filtered_df

In [ ]:
pro_df = processed_df1.loc[:,['open','high','low','close','vol']].reset_index(drop=True)

In [ ]:
pro_df

In [ ]:
#出现缺失值的个数、占比
count = len(pro_df[pro_df['open'] == 0])
nan_rate = count/len(pro_df)
print(count,'缺失值百分之:',nan_rate)

In [ ]:
#计算技术指标到dataset中
#Can be easily expanded
#Currently contains a small set of tech indicators
import talib as ta

def calc_tech_ind(data):
    #overlap 
    data['upbd'], data['midbd'], data['lowbd'] = ta.BBANDS(data["close"])
    data['dema'] = ta.DEMA(data["close"], timeperiod=30)
    data['tema'] = ta.TEMA(data["close"], timeperiod=30)
#     data['ema'] = ta.EMA(data["close"], timeperiod=30)
#     data['wma'] = ta.WMA(data["close"], timeperiod=30)
#     data['sma'] = ta.SMA(data["close"], timeperiod=30)
#     data['sarext'] = ta.SAREXT(data["high"], data["low"])
    
    #momentum
    data['adxr'] = ta.ADXR(data["high"], data["low"], data["close"], timeperiod=14)
    data['apo'] = ta.APO(data["close"], fastperiod=12, slowperiod=26, matype=0)
    data['aroondown'], data['aroonup'] = ta.AROON(data["high"], data["low"], timeperiod=14)
    data['cci'] = ta.CCI(data["high"], data["low"], data["close"], timeperiod=14)
    data['cmo'] = ta.CMO(data["close"], timeperiod=14)
    data['macd'], data['macdsignal'], data['macdhist'] = ta.MACD(data["close"], fastperiod=12, slowperiod=26, signalperiod=9)
    data['MFI'] = ta.MFI(data["high"], data["low"], data["close"], data['vol'], timeperiod=14)
#     data['mom'] = ta.MOM(data["close"], timeperiod=10)
#     data['plus_di'] = ta.PLUS_DI(data["high"], data["low"], data["close"], timeperiod=14)
#     data['ppo'] = ta.PPO(data["close"], fastperiod=12, slowperiod=26, matype=0)
#     data['roc'] = ta.ROC(data["close"], timeperiod=10)
#     data['rocp'] = ta.ROCP(data["close"], timeperiod=10)
#     data['rsi'] = ta.RSI(data["close"], timeperiod=14)
#     data['slowk'], data['slowd'] = ta.STOCH(data["high"], data["low"], data["close"])
#     data['fastk'], data['fastd'] = ta.STOCHF(data["high"], data["low"], data["close"])
#     data['trix'] = ta.TRIX(data["close"], timeperiod=30)
#     data['ultosc'] = ta.ULTOSC(data["high"], data["low"], data["close"], timeperiod1=7, timeperiod2=14, timeperiod3=28)
#     data['willr'] = ta.WILLR(data["high"], data["low"], data["close"], timeperiod=14)
    
    #volume
    data['ad'] = ta.AD(data["high"], data["low"], data["close"], data['vol'])
    data['obv'] = ta.OBV(data["close"], data['vol'])
    
    #volitility
    data['atr'] = ta.ATR(data["high"], data["low"], data["close"], timeperiod=14)
    data['natr'] = ta.NATR(data["high"], data["low"], data["close"], timeperiod=14)
    
    #cycle
    data['HT_DCPERIOD'] = ta.HT_DCPERIOD(data["close"])
#     data['HT_DCPHASE'] = ta.HT_DCPHASE(data["close"])
#     data['inphase'], data['quadrature'] = ta.HT_PHASOR(data["close"])
    
    
    return data

In [ ]:
full_list1 = calc_tech_ind(pro_df)
full_list1 = full_list1.fillna(0)

In [ ]:
full_list1

In [ ]:
full_list2 = calc_tech_ind(backtest_df_reduction)
full_list2 = full_list2.fillna(0)

In [ ]:
array_full_list = np.array(full_list1)
array_full_list = array_full_list.reshape(k,int(len(full_list1)/k),full_list1.shape[1])
selected_tics = sorted(selected_tics)

In [ ]:
len(array_full_list[1][3])

In [ ]:
len(array_full_list[:,:,3][0])

In [ ]:
array_full_list2 = np.array(full_list2)
array_full_list2 = array_full_list2.reshape(k,int(len(full_list2)/k),full_list2.shape[1])

In [ ]:
np.isnan(array_full_list).any()

In [ ]:
#text
# tdata = array_full_list[0][:-1]
ttdata = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
windows = 5
gap = 1
batch_size = 3
start = len(ttdata) - windows
test_list = []
while start >= 0:
#     print(ttdata[start:start + windows - 1],ttdata[start + windows-1])
    segdata = ttdata[start:start + windows]
#     segclose = ttdata[start:start + windows]
    test_list.extend([segdata] * batch_size)
#     test_list.append(segdata)
    start = start - gap
    print(test_list,'hhhhhhhhhhhhhhh',segdata)
    
# print(np.array(test_list)[::-1])

In [ ]:
# #generate x, y, z, zp quadruples
# #segment x, y, z trios to sequence according to $timeStep and $gap
# #x: historical data w/ technical analysis indicator
# #y: closing price of t+1
# #z:  difference between t+1 and t step's closing price

# def toSequential(idx, full_list, timeStep=48, gap=4):
#     #closing: from id=0 to last
#     closing=full_list[idx][:, 3]
#     #data from id=0 to second to last
#     data=full_list[idx][:-1]
#     #calculating number of available sequential samples
#     data_length=len(data)
#     count=(data_length-timeStep)//gap+1
#     stockSeq=[]
#     labelSeq=[]
#     diffSeq=[]
#     realDiffSeq=[]
    
#     start = data_length - timeStep
#     while start >= 0:
#         segData = data[start:start + timeStep - 1]
#         segClosing = closing[start:start + timeStep]
#         std_dev = segData.std(axis=0, keepdims=True)
#         std_dev_nonzero = np.where(std_dev == 0, 1, std_dev)  # 处理分母为零的情况
#         segDataNorm = np.nan_to_num((segData - segData.mean(axis=0, keepdims=True)) / std_dev_nonzero)
#         std_dev_close = segClosing.std()
#         std_dev_close_nonzero = np.where(std_dev_close == 0, 1, std_dev_close)
#         segClosingNorm=(segClosing-segClosing.mean())/std_dev_close_nonzero
#         stockSeq.append(segDataNorm)
#         labelSeq.append(segClosingNorm[1:])
# #         print(np.isnan(labelSeq).any())
#         diffSeq.append(segClosingNorm[1:]-segClosingNorm[:-1])
#         realDiffSeq.append(segClosing[1:]-segClosing[:-1])       
#         start = start - gap
#     stockSeq=np.array(stockSeq)[::-1]
#     labelSeq=np.array(labelSeq)[::-1]
#     diffSeq=np.array(diffSeq)[::-1]
#     realDiffSeq=np.array(realDiffSeq)[::-1]
    
#     return stockSeq.astype('float32') , labelSeq.astype('float32'), diffSeq.astype('float32'), realDiffSeq.astype('float32')

# Dataloader的构建

In [ ]:
# 输入是 50 * feature_size   full_list的形状是(full_date_range,k,feature_size)

def toSequential_train(full_list, seq_len = 48 , gap = 1):
    closing = full_list[:,:,3]
    data = full_list
    data_length = len(data)
    count = (data_length - seq_len) // gap + 1
    stockSeq=[]
    labelSeq=[]
    diffSeq=[]
    realDiffSeq=[]
    
    for i in range(count):
        #segData dims: [timestep, feature count]
        
        segData = data[gap * i: gap * i + seq_len]
        segClosing = closing[gap * i: gap * i + seq_len + 1]
        
        #segDiff=diff[gap*i:gap*i+timeStep]
        #normalization
        
        std_dev = segData.std(axis=0, keepdims=True)
        std_dev_nonzero = np.where(std_dev == 0, 1, std_dev)  # 处理分母为零的情况
        
#         segDataNorm = np.nan_to_num((segData - segData.mean(axis=0, keepdims=True)) / std_dev_nonzero)
        segDataNorm = np.nan_to_num((segData-segData.mean(axis=0, keepdims=True))/segData.std(axis=0, keepdims=True))

        std_close = segClosing.std()
        std_segClosing = np.where(std_close == 0, 1, std_close)
        segClosingNorm=(segClosing-segClosing.mean())/std_segClosing
#         segDiff=(segDiff-segDiff.mean())/segDiff.std()
        
        stockSeq.append(segDataNorm)
        labelSeq.append(segClosingNorm[1:])
        diffSeq.append(segClosingNorm[1:]-segClosingNorm[:-1])
        realDiffSeq.append(segClosing[1:]-segClosing[:-1])
        print(len(labelSeq))
    stockSeq=np.array(stockSeq)
    labelSeq=np.array(labelSeq)
    diffSeq=np.array(diffSeq)
    realDiffSeq=np.array(realDiffSeq)
    return stockSeq.astype('float32') , labelSeq.astype('float32'), diffSeq.astype('float32'), realDiffSeq.astype('float32')

In [ ]:
def toSequential_back(full_list, seq_len = 48, gap = 1):
    closing = full_list[:,:,3]
    data = full_list
    data_length = len(data)
    count = (data_length - seq_len) // gap + 1
    stockSeq=[]
    labelSeq=[]
    diffSeq=[]
    realDiffSeq=[]
    
    start = data_length - seq_len
    i=0
    while start >= 0:
        segData = data[start:start + seq_len]
        std_dev = segData.std(axis=0, keepdims=True)
        std_dev_nonzero = np.where(std_dev == 0, 1, std_dev)  # 处理分母为零的情况
        segDataNorm = np.nan_to_num((segData - segData.mean(axis=0, keepdims=True)) / std_dev_nonzero)
        #复制batch_size个
        stockSeq.append(segDataNorm)
#         stockSeq.extend([segDataNorm] * batch_size)
#         stockSeq.append(segDataNorm)   
        start = start - gap
        i +=1
    stockSeq=np.array(stockSeq)[::-1]
#     print(i)
    return stockSeq.astype('float32')

In [ ]:
#input each step:  vector including [stock info, tech indicators]
#output each step: closing price t+1, price diff between t+1 and t
#full_list: output from get_data_set

class StockDataset_back(Dataset):
    def __init__(self, full_list, seq_len = 48, gap = 1):

        X = toSequential_back(full_list, seq_len = seq_len, gap = gap)

        self.X = X

        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):

        data=self.X[index]
        label1=self.y[index]

        return (data, label1)

In [ ]:
#input each step:  vector including [stock info, tech indicators]
#output each step: closing price t+1, price diff between t+1 and t
#full_list: output from get_data_set

class StockDataset_train(Dataset):
    def __init__(self, split_rate, full_list, seq_len = 48, gap=3):

        X, y, z, zp=toSequential_train(full_list, seq_len = seq_len, gap = gap)

        self.X = X[:int(len(X)*split_rate),:,:]
        self.y = y[:int(len(y)*split_rate),:]
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):

        data=self.X[index]
        label1=self.y[index]

        return (data, label1)

In [ ]:
class StockDataset_val(Dataset):
    def __init__(self, split_rate, full_list, seq_len=24, gap=1):

        X, y, z, zp=toSequential_train(full_list, seq_len = seq_len, gap = gap)

        self.X = X[int(len(X)*split_rate):,:,:]
        self.y = y[int(len(y)*split_rate):,:]
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):

        data=self.X[index]
        label1=self.y[index]

        return (data, label1)

In [ ]:

val_iter = DataLoader(StockDataset_val(split_rate = 0.7,full_list = array_full_list), shuffle=True, batch_size=64, num_workers=0,drop_last=True)
train_iter = DataLoader(StockDataset_train(split_rate = 0.7,full_list = array_full_list), shuffle=True, batch_size=64, num_workers=0,drop_last=True)
back_iter = DataLoader(StockDataset_back(full_list = array_full_list), shuffle=False, batch_size=64, num_workers=0,drop_last=True)


In [ ]:
#generate x, y, z, zp quadruples
#segment x, y, z trios to sequence according to $timeStep and $gap
#x: historical data w/ technical analysis indicator
#y: closing price of t+1
#z:  difference between t+1 and t step's closing price

def toSequential_train1(idx, full_list, timeStep=48, gap=4):
    #closing: from id=0 to last
    closing=full_list[idx][:, 3]
#     closingNorm = (closing - closing.mean())/closing.std()
    #data from id=0 to second to last
    data=full_list[idx][:-1]
    #calculating number of available sequential samples
    data_length=len(data)
    count=(data_length-timeStep)//gap+1
    stockSeq=[]
    labelSeq=[]
    diffSeq=[]
    realDiffSeq=[]
    for i in range(count):
        #segData dims: [timestep, feature count]       
        segData=data[gap*i:gap*i+timeStep]
        segClosing=closing[gap*i:gap*i+timeStep+1]
        #segDiff=diff[gap*i:gap*i+timeStep]
        #normalization
        
        std_dev = segData.std(axis=0, keepdims=True)
        std_dev_nonzero = np.where(std_dev == 0, 1, std_dev)  # 处理分母为零的情况
#         segDataNorm = np.nan_to_num((segData - segData.mean(axis=0, keepdims=True)) / std_dev_nonzero)
        segDataNorm=np.nan_to_num((segData-segData.mean(axis=0, keepdims=True))/segData.std(axis=0, keepdims=True))
    
        std_close = segClosing.std()
        std_segClosing = np.where(std_close == 0, 1, std_close)
        segClosingNorm=(segClosing-segClosing.mean())/std_segClosing
#         segDiff=(segDiff-segDiff.mean())/segDiff.std()
        
        stockSeq.append(segDataNorm)
        labelSeq.append(segClosingNorm[1:])
        diffSeq.append(segClosingNorm[1:]-segClosingNorm[:-1])
        realDiffSeq.append(segClosing[1:]-segClosing[:-1])
    stockSeq=np.array(stockSeq)
    labelSeq=np.array(labelSeq)
    diffSeq=np.array(diffSeq)
    realDiffSeq=np.array(realDiffSeq)
    return stockSeq.astype('float32') , labelSeq.astype('float32'), diffSeq.astype('float32'), realDiffSeq.astype('float32')

In [ ]:
#input each step:  vector including [stock info, tech indicators]
#output each step: closing price t+1, price diff between t+1 and t
#full_list: output from get_data_set

class StockDataset(Dataset):
    def __init__(self, id_list, full_list, transform=None, timestep=48, gap=3):
        self.transform=transform
        self.id_list=id_list
        
        stock_cohort=[]
        closing_cohort=[]
        diff_cohort=[]
        real_diff_cohort=[]
        
        #load data into cohort
        for i in self.id_list:
            X, y, z, zp=toSequential_train1(i, full_list, timeStep=timestep, gap=gap)
            stock_cohort.append(X)
            closing_cohort.append(y)
            diff_cohort.append(z)
            real_diff_cohort.append(zp)
        self.X=np.concatenate(stock_cohort, axis=0)
        self.y=np.concatenate(closing_cohort, axis=0)
        self.z=np.concatenate(diff_cohort, axis=0)  
        self.zp=np.concatenate(real_diff_cohort, axis=0)
        print(self.X)
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        """
        data returned in the format of 
        """
        if torch.is_tensor(idx):
            idx=idx.tolist()
        
        data=self.X[idx]
        label1=self.y[idx]
        label2=self.z[idx]
        label3=self.zp[idx]
        if self.transform:
            data=self.transform(data)
        return (data, label1, label2, label3)
    
    
    def getDS(self):
        return self.X, self.y, self.z, self.zp

In [ ]:
#generate x, y, z, zp quadruples
#segment x, y, z trios to sequence according to $timeStep and $gap
#x: historical data w/ technical analysis indicator
#y: closing price of t+1
#z:  difference between t+1 and t step's closing price

def toSequential_train(idx, full_list, timeStep=48, gap=4):
    #closing: from id=0 to last
    closing=full_list[idx][:, 3]
    closingNorm = (closing - closing.mean())/closing.std()
    #data from id=0 to second to last
    data=full_list[idx][:-1]
    #calculating number of available sequential samples
    data_length=len(data)
    count=(data_length-timeStep)//gap+1
    stockSeq=[]
    labelSeq=[]
    diffSeq=[0,]
    realDiffSeq=[0,]
    for i in range(count-1):
        #segData dims: [timestep, feature count]       
        segData=data[gap*i:gap*i+timeStep]
        segClosing=closingNorm[gap*i+timeStep]
        #segDiff=diff[gap*i:gap*i+timeStep]
        #normalization
        
        std_dev = segData.std(axis=0, keepdims=True)
        std_dev_nonzero = np.where(std_dev == 0, 1, std_dev)  # 处理分母为零的情况
        segDataNorm = np.nan_to_num((segData - segData.mean(axis=0, keepdims=True)) / std_dev_nonzero)
#       segDataNorm=np.nan_to_num((segData-segData.mean(axis=0, keepdims=True))/segData.std(axis=0, keepdims=True))
#       segClosingNorm=(segClosing-segClosing.mean())/segClosing.std()
        #segDiff=(segDiff-segDiff.mean())/segDiff.std()
        
        stockSeq.append(segDataNorm)
        labelSeq.append(segClosing)
        diffSeq.append(segClosing-diffSeq[-1])
        realDiffSeq.append(closing[gap*i+timeStep]-realDiffSeq[-1])
    stockSeq=np.array(stockSeq)
    labelSeq=np.array(labelSeq)
    diffSeq=np.array(diffSeq)
    realDiffSeq=np.array(realDiffSeq)
    return stockSeq.astype('float32') , labelSeq.astype('float32'), diffSeq.astype('float32'), realDiffSeq.astype('float32')

In [ ]:
def toSequential_back(idx, full_list, timeStep = 48, gap = 1,batch_size = 64):
    #closing: from id=0 to last
    closing=full_list[idx][:, 3]
    #data from id=0 to second to last
    data=full_list[idx][:-1]
    #calculating number of available sequential samples
    data_length=len(data)
    count=(data_length-timeStep)//gap+1
    stockSeq=[]
    labelSeq=[]
    diffSeq=[]
    realDiffSeq=[]
    
    start = data_length - timeStep
    i=0
    while start >= 0:
        segData = data[start:start + timeStep]
        std_dev = segData.std(axis=0, keepdims=True)
        std_dev_nonzero = np.where(std_dev == 0, 1, std_dev)  # 处理分母为零的情况
        segDataNorm = np.nan_to_num((segData - segData.mean(axis=0, keepdims=True)) / std_dev_nonzero)
        #复制batch_size个
        stockSeq.extend([segDataNorm] * batch_size)
#         stockSeq.append(segDataNorm)   
        start = start - gap
        i +=1
    stockSeq=np.array(stockSeq)[::-1]
#     print(i)
    return stockSeq.astype('float32')

In [ ]:
#input each step:  vector including [stock info, tech indicators]
#output each step: closing price t+1, price diff between t+1 and t
#full_list: output from get_data_set

class StockDataset_back(Dataset):
    def __init__(self, id_list, full_list, transform=None, timestep=48, gap=1):
        self.transform=transform
        self.id_list=id_list
        
        stock_cohort=[]
        
        #load data into cohort
        for i in self.id_list:
            X=toSequential_back(i, full_list, timeStep=timestep, gap=gap)
            stock_cohort.append(X)
        self.X=np.concatenate(stock_cohort, axis=0)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        """
        data returned in the format of 
        """
        if torch.is_tensor(idx):
            idx=idx.tolist()
        
        data=self.X[idx]
        if self.transform:
            data=self.transform(data)
        return data
        
    def getDS(self):
        return self.X

In [ ]:
#Generation of training, validation, and testing dataset
def DataIterGen(test_id_list, val_id_list, name_list, full_list, demo=False):
    """
    test_id_list: id of subjects for testing
    val_id_list: id of subjects for validation
    other subjects for training
    full_list=get_data_set(name_list), preprocessed
    demo: when demo mode is True, only test_iter is returned, with data from
    first entry of test_id_list (single stock)
    """
    name_count=len(name_list)

    if demo:
        test_iter=DataLoader(StockDataset(test_id_list[0:1], full_list, timestep=24, gap=1), shuffle=False, batch_size=64, num_workers=0)
        print(f'Demo with stock: {name_list[test_id_list[0]]} ')
        return test_iter
    else:
        all_ids = list(range(name_count))
        train_id_list = list(set(all_ids) - set(test_id_list) - set(val_id_list))
#         partial_list=full_list[train_list,:,:]
        test_iter=DataLoader(StockDataset(test_id_list, full_list), batch_size=64, num_workers=0,drop_last=True)
        val_iter=DataLoader(StockDataset_back(val_id_list, full_list), batch_size=64, num_workers=0,drop_last=True)
        train_iter=DataLoader(StockDataset(train_id_list, full_list), shuffle=True, batch_size=64, num_workers=0,drop_last=True)
        print(f'Val: {[name_list[val_id] for val_id in val_id_list]}, Test: {[name_list[test_id] for test_id in test_id_list]}, Train: {[name_list[train_id] for train_id in train_id_list]} ')
        return train_iter, val_iter, test_iter

In [ ]:
mm = [1,2,3,4,5,6,7]
rate = 0.7
print(mm[:int(len(mm)*rate)],mm[int(len(mm)*rate):])

In [ ]:
train_iter, val_iter, test_iter = DataIterGen([3,4,5,9,10],[7,11,13,14],selected_tics,array_full_list)

In [ ]:
len(array_full_list[0][0])

# Transformer

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Transformer(nn.Module):
    def __init__(self, input_size, d_model, d_ff, num_heads,env_size, num_layers,dropout_rate):
        super(Transformer, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.encode = Encoder(d_model, d_ff, num_heads, num_layers,dropout_rate)
        self.linear1 = nn.Linear(d_model, env_size)
        self.rule = nn.ReLU()
        self.linear2 = nn.Linear(env_size,1)
        

    def forward(self, x):
        x = self.embedding(x)
#         print(x.shape)
        x = x.permute(1,0,2)
        encoded = self.encode(x)
        encoded = self.linear1(encoded)
        x = self.rule(encoded)
        x = self.linear2(x)
#         print(x.shape)
        decoded = x.permute(1,0,2)
        return decoded, encoded.permute(1,0,2)[0,-1,:]  


In [ ]:
class Encoder(nn.Module):
    def __init__(self, d_model, d_ff, num_heads, num_layers,dropout_rate):
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, d_ff, num_heads, dropout_rate) for _ in range(num_layers)])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, d_ff, num_heads,dropout_rate):
        super(EncoderLayer, self).__init__()
        self.self_attention = nn.MultiheadAttention(d_model, num_heads)
#         self.dropout1 = nn.Dropout(dropout_rate)
        self.feed_forward = FeedForward(d_model, d_ff, dropout_rate)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        residual = x  # 保存输入的残差连接
        encoder_mask = torch.triu(torch.ones(x.size(0), x.size(0)), diagonal=1).bool().to(device)
        # Self-Attention
        x, _ = self.self_attention(x, x, x,attn_mask=encoder_mask)
#         x = self.dropout1(x)
        x = x + residual  # 残差连接
        x = self.norm1(x)  # Add & Norm

        residual = x  # 保存 Self-Attention 后的残差连接

        # Feed-Forward
        x = self.feed_forward(x)
        x = x + residual  # 残差连接
        x = self.norm2(x)  # Add & Norm

        return x

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout_rate):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.dropout1(x)
        x = self.linear2(x)
        return x

In [ ]:
#模型训练
def train(model, train_iter, optimizer, num_epochs): 
    # 训练循环
    train_losses = []
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        enVec_list = []
        for X, y, z, zp in train_iter:  
            optimizer.zero_grad()

            inputs = X.to(device)
            targets = y.unsqueeze(2).to(device)
            pred,enVec = model(inputs)
            enVec_list.append(enVec)
#             print(pred,targets)
            loss = nn.MSELoss()(pred, targets)  
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        train_losses.append(total_loss/len(train_iter))
        # 打印每个 epoch 的损失
        print(f"Epoch {epoch+1}: Loss: {total_loss/len(train_iter):.4f}")
    %matplotlib inline    
        # 绘制损失变化曲线
    plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.show()
    
    return enVec_list

# 模型评估
def val(model, val_iter):
    model.eval()
    total_loss = 0.0
    enVec_list = []
    with torch.no_grad():
        for X, y, z, zp in val_iter: 
            inputs = X.to(device)
            targets = y.unsqueeze(2).to(device)
            pred, enVec = model(inputs)
            enVec_list.append(enVec)
            loss = nn.MSELoss()(pred, targets)  
            total_loss += loss.item()
    print(f"val Loss: {total_loss/len(val_iter):.4f}")
    return enVec_list

In [ ]:
#模型训练
def train(model, train_iter, optimizer, num_epochs): 
    # 训练循环
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        enVec_list = []
        predictions = []  # 保存每个epoch的一些样本的预测值
        targets_list = []  # 保存每个epoch的一些样本的真实值
        for X, y, z, zp in train_iter:  
            optimizer.zero_grad()

            inputs = X.to(device)
            targets = y.unsqueeze(2).to(device)
            pred, enVec = model(inputs)
            enVec_list.append(enVec)
            
            # 保存一些样本的预测值和真实值
            if np.random.rand() < 0.1:  # 以10%的概率保存样本
                predictions.append(pred.cpu().detach().numpy())
                targets_list.append(targets.cpu().detach().numpy())
            
            loss = nn.MSELoss()(pred, targets)  
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # 打印每个 epoch 的损失
        print(f"Epoch {epoch+1}: Loss: {total_loss/len(train_iter):.4f}")
        
        # 绘制一些样本的预测值与真实值图
        if epoch % 5 == 0:  # 每5个epoch绘制一次
            plot_predictions(targets_list, predictions, epoch)

    return enVec_list

# 模型评估
def val(model, val_iter):
    model.eval()
    total_loss = 0.0
    enVec_list = []
    with torch.no_grad():
        for X, y, z, zp in val_iter: 
            inputs = X.to(device)
            targets = y.unsqueeze(2).to(device)
            pred, enVec = model(inputs)
            enVec_list.append(enVec)
            loss = nn.MSELoss()(pred, targets)  
            total_loss += loss.item()
    print(f"val Loss: {total_loss/len(val_iter):.4f}")
    return enVec_list

In [ ]:
def plot_predictions(targets_list, predictions, epoch):
    targets = np.concatenate(targets_list, axis=0)
    predictions = np.concatenate(predictions, axis=0)
    
    plt.figure(figsize=(10, 6))
    for i in range(min(5, targets.shape[0])):  # 绘制前5个样本
        plt.subplot(5, 1, i+1)
        plt.plot(targets[i], label='True')
        plt.plot(predictions[i], label='Predicted')
        plt.title(f'Sample {i+1} - Epoch {epoch+1}')
        plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
def predict(model,back_iter):
    enVec_list = []
    with torch.no_grad():
        for X in back_iter:
            inputs = X.to(device)
            _,enVec = model(inputs)
            enVec_list.append(enVec)
    return enVec_list

In [ ]:
#define device
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
device=try_gpu()

In [ ]:
# 定义模型
model = Transformer(input_size = 25, d_model = 64 , num_layers = 2, d_ff = 128 ,num_heads = 8,env_size =20,dropout_rate=0.2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
outcoming =train(model,train_iter,optimizer,6)
valtest =val(model,val_iter)

In [ ]:
# feature_enginner
import time 
start_time = time.time()
alltic_list = [i for i in range(k)]

def Feature_enginner(model, processed_df, full_list, alltic_list, batch_size=64, k=20, env_size=40,dela = 48):
    back_iter = DataLoader(StockDataset_back([i for i in range(k)],full_list), shuffle=False, batch_size=batch_size, num_workers=0,drop_last=False)
    
    new_state = predict(model,back_iter)
    print(1)
    concatenated_tensor = torch.cat(new_state, dim=0)
    print(2)
    concatenated_tensor = concatenated_tensor.view(-1,env_size)
    print(3)
    feature_df = pd.DataFrame(concatenated_tensor.numpy())
    print(4)
    
    full_delay_df = delaydate(dela ,processed_df)
    
    merged_df = pd.concat([full_delay_df, feature_df], axis=1)
    merged_df = merged_df.loc[:,['tic','date','close'] + INDICATORS + [i for i in range(env_size)]]
    merged_df.columns = ['tic','date','close'] + INDICATORS + [f"temporal_feature_{i}" for i in range(env_size)]
    end_time = time.time()
    print('总计消耗时间:',(end_time - start_time)/60)
    return merged_df

In [ ]:
addfeature_df = Feature_enginner(model, processed_df1, array_full_list, alltic_list, batch_size=64, k=50 ,env_size=20, dela=48)

In [28]:
addfeature_df

,tic,date,close,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,...,temporal_feature_10,temporal_feature_11,temporal_feature_12,temporal_feature_13,temporal_feature_14,temporal_feature_15,temporal_feature_16,temporal_feature_17,temporal_feature_18,temporal_feature_19
0,600000.SH,2017-03-17,16.20,-0.103361,16.920131,16.024869,42.451553,-109.290506,24.313216,16.560333,...,1.500900,-0.444595,2.496382,1.498984,-1.503451,-0.268903,1.428747,0.559653,-1.066029,2.959662
1,600000.SH,2017-03-20,16.14,-0.110808,16.854846,16.013154,40.998417,-121.936983,25.144384,16.543000,...,1.900961,-0.317013,2.691144,2.069022,-1.325521,0.301127,1.796261,-0.097198,-1.664815,3.305223
2,600000.SH,2017-03-21,16.00,-0.126338,16.798334,15.981666,37.869445,-145.491419,36.351888,16.520667,...,2.183464,-0.027345,2.698100,2.387536,-1.106515,0.843225,1.773876,-0.257767,-1.632134,3.319397
3,600000.SH,2017-03-22,15.78,-0.154311,16.797444,15.885556,33.689702,-193.198499,50.992976,16.491000,...,2.220645,0.032896,2.916779,2.445098,-1.038702,0.922410,1.836789,-0.038941,-1.385233,3.098996
4,600000.SH,2017-03-23,15.88,-0.166606,16.770306,15.831694,36.961361,-156.439114,50.992976,16.463000,...,1.768891,-0.994125,2.443021,2.249985,-0.970342,-1.083996,1.311004,-1.812655,-2.678623,4.134655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70545,601998.SH,2022-12-26,4.83,0.031323,5.190485,4.833515,50.959756,-36.727817,0.176333,4.928000,...,-0.779195,-0.934460,1.517315,-1.088990,-0.973530,-1.664009,0.232380,2.426849,-0.475797,-0.119264
70546,601998.SH,2022-12-27,4.90,0.025253,5.174477,4.825523,52.808796,-19.119878,2.625621,4.935333,...,-0.734905,-0.892610,1.788705,-0.985208,-1.021302,-1.598934,0.435053,2.181200,-0.728077,0.176656
70547,601998.SH,2022-12-28,4.98,0.026590,5.171599,4.823401,54.822659,8.610136,14.173296,4.946667,...,-1.004894,-0.915082,1.391061,-1.209658,-0.966595,-1.683521,0.205795,2.563425,-0.428341,-0.234574
70548,601998.SH,2022-12-29,4.93,0.023347,5.171576,4.818424,53.350646,-44.954584,9.544348,4.957333,...,-1.267627,-0.967534,0.878120,-1.595519,-0.918358,-1.892846,-0.190184,2.897044,-0.247745,-0.814026


In [ ]:
merged_df1

In [ ]:
addfeature_df.to_csv('1227_addfeature_df.csv',index=None)

In [27]:
addfeature_df = pd.read_csv('1227_addfeature_df.csv')

In [ ]:
merged_df = pd.read_csv('1228_merged.csv')

In [ ]:
merged_df['date'] = pd.to_datetime(merged_df['date'])

In [ ]:
addfeature_df['date'] = pd.to_datetime(addfeature_df['date'])

In [ ]:
merged_df1 = merged_df[['trade_date','ts_code','filled','ema']]
merged_df1['date'] = pd.to_datetime(merged_df1['trade_date'])
merged_df1['tic'] = merged_df1['ts_code']
merged_df1 = merged_df1.loc[:,('date','tic','filled','ema')]

In [ ]:
addfeature_df1 = pd.merge(addfeature_df,merged_df1,on=('date','tic'),how='left')
addfeature_df1

In [ ]:
processed_df3.to_csv('1227_processed.csv',index=None)

In [ ]:
processed_df3

In [ ]:
processed_df3 = pd.read_csv('1227_processed.csv')

In [ ]:
processed_df3['date'] = pd.to_datetime(processed_df3['date'])

In [ ]:
addfeature_df[addfeature_df['tic'] == '601939.SH']

In [ ]:
torch.triu(g,diagonal=1+3)

In [ ]:
torch.triu(g,diagonal=1)

<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [ ]:
TRAIN_START_DATE = '2015-03-02'
TRAIN_END_DATE = '2020-07-30'
TRADE_START_DATE = '2020-07-31'
TRADE_END_DATE = '2021-02-26'

In [ ]:
TRAIN_START_DATE = addfeature_df['date'].unique()[0]

In [ ]:
train = data_split(addfeature_df1, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(addfeature_df1, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

In [ ]:
addfeature_df

In [ ]:
train.tail(10)

In [ ]:
selfattn_indicator_list = [f"temporal_feature_{i}" for i in range(20)]

In [ ]:
selfattn_indicator_list

In [ ]:
trade.head()

In [ ]:
INDICATORS

In [ ]:
# stock_dimension = len(train.tic.unique())
# state_space = 1 + 2*stock_dimension + len(selfattn_indicator_list)*stock_dimension
# print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 4*stock_dimension + len(INDICATORS)*stock_dimension + len(selfattn_indicator_list)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


In [ ]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 500,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "risk_preference": 'ne',   #ne为中性、pr为激进,除此之外都是保守策略
    'adjust': True,  #True为启用动态调整机制，False关闭
    "reward_scaling": 1,
    "attn_indicator_list":selfattn_indicator_list,
    "alpha": 100, #风险喜好函数系数
    "beta" : 100, #风险中性函数系数
    "c" : 100, # 风险厌恶函数系数
    "theta" : 0.2, # 风险喜好放缩因子
    "gamma" : 5, # 风险厌恶放缩因子
    "mu" : 1, #惩罚项放缩因子
    "p_lambda" : 100,  #惩罚系数
    'up_ad_thre': 0.15, #动态调整机制中向上变化率阈值
    'down_ad_thre': 0.1,  #动态调整机制中向下变化率阈值
    
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)


## Environment for Training



In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [ ]:
agent = DRLAgent(env = env_train)

if_using_a2c = False
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = True
if_using_sac = False


In [ ]:
import torch
device = torch.device("mps")

### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [ ]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)


In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

### Agent 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

### Agent 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=30000) if if_using_ppo else None

### Agent 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000) if if_using_td3 else None

### Agent 5: SAC

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

In [ ]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000) if if_using_sac else None

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
data_risk_indicator = addfeature_df[(addfeature_df.date<TRAIN_END_DATE) & (addfeature_df.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:
insample_risk_indicator.vix.describe()

In [ ]:
insample_risk_indicator.vix.quantile(0.996)

In [ ]:
insample_risk_indicator.turbulence.describe()

In [ ]:
insample_risk_indicator.turbulence.quantile(0.996)

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
e_trade_gym = StockTradingEnv(df = trade,**env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trade.head()

In [ ]:
trained_moedl = trained_td3
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

In [ ]:
df_account_value.to_csv('107_up_up_df_account_value.csv',index=None)

In [ ]:
df_account_value

In [ ]:
df_account_value.tail()

In [ ]:
df_actions.head()

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
#now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
#perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
print("==============Get Backtest Results===========")
#now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
#perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
print("==============Get Backtest Results===========")
#now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
#perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [24]:
#获取随机选择的tic在select_date的权重并修正
def get_selected_index_weight(df_index,select_date,selected_tics):
    df = df_index[df_index['trade_date'] == select_date]
    select_df = df[df['con_code'].isin(selected_tics)]
    select_df = select_df.drop('index_code',axis=1).rename(columns={'con_code':'tic','trade_date':'date'})
    select_df = select_df.reset_index()
    weight_sum = select_df['weight'].sum()
#     print(select_df)
    for i in range(len(select_df)):
        select_df.loc[i,'weight'] = (select_df.loc[i,'weight'] / weight_sum)*100
    return select_df.loc[:,['tic','date','weight']]

baseline_weight =get_selected_index_weight(df_index,select_date,selected_tics)
baseline_weight = baseline_weight.sort_values(['tic'],ascending=True).reset_index(drop=True)
print(baseline_weight)

          tic      date    weight
0   600000.SH  20160229  5.216948
1   600010.SH  20160229  0.813992
2   600015.SH  20160229  1.507985
3   600016.SH  20160229  7.570924
4   600018.SH  20160229  0.455995
5   600028.SH  20160229  1.385986
6   600030.SH  20160229  3.407966
7   600036.SH  20160229  4.552954
8   600048.SH  20160229  1.776982
9   600050.SH  20160229  1.032990
10  600104.SH  20160229  1.837982
11  600109.SH  20160229  0.625994
12  600111.SH  20160229  0.735993
13  600150.SH  20160229  0.437996
14  600518.SH  20160229  1.217988
15  600519.SH  20160229  3.273967
16  600585.SH  20160229  0.853991
17  600637.SH  20160229  1.371986
18  600795.SH  20160229  0.863991
19  600837.SH  20160229  3.041970
20  600887.SH  20160229  2.388976
21  600893.SH  20160229  0.827992
22  600958.SH  20160229  0.552994
23  600999.SH  20160229  1.229988
24  601006.SH  20160229  1.169988
25  601088.SH  20160229  0.817992
26  601166.SH  20160229  5.733943
27  601169.SH  20160229  2.953970
28  601186.SH 

In [25]:
#计算所选股票的buy & hold策略
def calculate_selected_baseline(df,full_date_range,selected_tics):
    df = df.sort_values(['tic','date'],ascending=True).reset_index(drop=True)
    baseline = pd.DataFrame({'date':full_date_range['date']})
    for i in range(len(full_date_range)):
        temp_date = full_date_range.loc[i,'date']
        temp_df = df[df['date'] == temp_date].sort_values('tic',ascending=True)
        close = list(temp_df['close'])
        weight = list(baseline_weight['weight'])
        baseline.loc[i,'account_value'] = sum(np.array(close) * np.array(weight)) #注意权重和收盘价对应的股票顺序
    baseline['date'] = pd.to_datetime(baseline['date'],format='%Y%m%d')
#     baseline.set_index("date", inplace=True, drop=True)
#     baseline.index = baseline.index.tz_localize("UTC")
    return baseline
# full_date_range = get_trading_days(exchange='SSE',start_date='20210301', end_date='20230227')
# full_date_range = full_date_range.sort_values('trade_date',ascending=True).reset_index(drop=True)
# subset_df = addfeature_df.loc[(addfeature_df['date'].astype(str) >='20210301') & (addfeature_df['date'].astype(str)<='20220227')]
full_date_range1 = pd.DataFrame({'date':df_account_value['date'].unique()}).reset_index(drop=True)
baseline_sse = calculate_selected_baseline(addfeature_df,full_date_range1,selected_tics)
baseline_sse

NameError: name 'df_account_value' is not defined

In [ ]:
addfeature_df

In [ ]:
baseline_sse.info()

In [ ]:
# baseline_sse = pro.index_daily(ts_code='000016.sh',start_date = '20200301',end_date='20230227')
# baseline_sse = baseline_sse.rename(columns={'trade_date':'date'})
# baseline_sse = baseline_sse.sort_values('date',ascending=True)
# print(baseline_sse)

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
# baseline_df = get_baseline(
#         ticker="^DJI", 
#         start = df_account_value.loc[0,'date'],
#         end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_sse, value_col_name = 'account_value')


In [ ]:
df_account_value

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
import pyfolio
from copy import deepcopy
def backtest_plot_com(
    account_value,
    baseline,
    baseline_start=TRADE_START_DATE,
    baseline_end=TRADE_END_DATE,
    value_col_name="account_value",
):
    df = deepcopy(account_value)
    df["date"] = pd.to_datetime(df["date"])
    test_returns = get_daily_return(df, value_col_name=value_col_name)
#     pro = ts.pro_api()
#     baseline_df = pro.index_daily(ts_code='000016.sh',start_date = '20200301',end_date='20230227')
#     baseline_df = baseline_df.rename(columns={'trade_date':'date'})
#     baseline_df = baseline_df.sort_values('date',ascending=True)
    baseline_returns = get_daily_return(baseline, value_col_name=value_col_name)
    with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(
            returns=test_returns, benchmark_rets=baseline_returns, set_context=False
        )

In [ ]:
print("==============Compare to SSE50===========")
# from finrl.plot import backtest_plot_com
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot_com(account_value = df_account_value, 
                  baseline = baseline_sse,
                  baseline_start = df_account_value.loc[len(df_account_value)-1,'date'],
                  baseline_end = df_account_value.loc[0,'date']
                     )

In [ ]:
print("==============Compare to SSE50===========")
# from finrl.plot import backtest_plot_com
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot_com(account_value = df_account_value, 
                  baseline = baseline_sse,
                  baseline_start = df_account_value.loc[len(df_account_value)-1,'date'],
                  baseline_end = df_account_value.loc[0,'date']
                     )

In [ ]:
print("==============Compare to SSE50===========")
# from finrl.plot import backtest_plot_com
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot_com(account_value = df_account_value, 
                  baseline = baseline_sse,
                  baseline_start = df_account_value.loc[len(df_account_value)-1,'date'],
                  baseline_end = df_account_value.loc[0,'date']
                     )

In [ ]:
print(df_account_value.loc[len(df_account_value)-1,'date'],df_account_value.loc[0,'date'])